In [1]:
from dgl.data.utils import save_graphs
import dgl
import torch as th
import scipy.sparse as sp
import networkx as nx
import pickle
from numpy import array
import pandas as pd

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#DataFrame
import pandas as pd

#Rimozione duplicati lista
from collections import OrderedDict

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#Salvataggio dizinario
import pickle 

from sklearn import preprocessing

import csv

In [2]:
path = "../../R/Rdati/Matrici Adiacenza GNN_Binaria/"

In [3]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

Amino = []
ID_Graph = []
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            
            id_p = filename[0:4]
            ID_Graph.append(id_p)
            matrix = "../../R/Rdati/Matrici Adiacenza GNN_Binaria/{}.csv".format(id_p)            
            proteina = pd.read_csv(matrix, sep = ';')
            Amino.extend(list(proteina.iloc[0][1:]))


Amino_TOT = list(OrderedDict.fromkeys(Amino))

classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

if not exists("../Dati/Dataset_Binario/aminoacidi.pickle"):
    aminoAcidi = preprocessing.LabelEncoder()
    aminoAcidi.fit(Amino_TOT)
    f = open("../Dati/Dataset_Binario/aminoacidi.pickle","wb")
    pickle.dump(aminoAcidi, f)
    f.close()

if not exists("../Dati/Dataset_Binario/classi.pickle"):
    classi = preprocessing.LabelEncoder()
    classi.fit(classes)
    f = open("../Dati/Dataset_Binario/classi.pickle","wb")
    pickle.dump(classi, f)
    f.close()

if not exists("../Dati/Dataset_Binario/ID_Graphs.pickle"):
    ID_Graphs = preprocessing.LabelEncoder()
    ID_Graphs.fit(ID_Graph)
    f = open("../Dati/Dataset_Binario/ID_Graphs.pickle","wb")
    pickle.dump(ID_Graphs, f)
    f.close()

In [4]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

f = open("../Dati/Dataset_Binario/classi.pickle","rb")
classi = pickle.load(f)
f.close()

f = open("../Dati/Dataset_Binario/aminoacidi.pickle","rb")
amino = pickle.load(f)
f.close()

f = open("../Dati/Dataset_Binario/ID_Graphs.pickle","rb")
ID_Graphs_read = pickle.load(f)
f.close()

ID_Graph = []

header_edges = ["graph_id", "src_id", "dst_id"]
f_e = open("../Dati/Dataset_Binario/edges.csv", 'w')
writer_e = csv.writer(f_e)
writer_e.writerow(header_edges)

header_nodes = ["graph_id", "node_id", "label"]
f_n = open("../Dati/Dataset_Binario/nodes.csv", 'w')
writer_n = csv.writer(f_n)
writer_n.writerow(header_nodes)

header_graphs = ["graph_id", "label"]
f_g = open("../Dati/Dataset_Binario/graphs.csv", 'w')
writer_g = csv.writer(f_g)
writer_g.writerow(header_graphs)

#edges = pd.DataFrame(columns=["graph_id", "src_id", "dst_id"])

#nodes = pd.DataFrame(columns=["graph_id", "node_id", "label"])

#graphs = pd.DataFrame(columns=["graph_id", "label"])


for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            id_p = filename[0:4]
            #print(id_p)

            #Id della proteina che si sta analizzando
            print(id_p)

            if id_p not in ID_Graph: 
                ID_Graph.append(id_p)
                matrix = "../../R/Rdati/Matrici Adiacenza GNN_Binaria/{}.csv".format(id_p)
                #print(matrix)

                #Matrice adiacenza singola matrice inserita in un dataframe
                proteina = pd.read_csv(matrix, sep = ';')
                #Nomi dei nodi nonché numero
                Nodes = list(proteina.columns[1:])
                
                for vertex in range(0,len(Nodes)):   
                    Nodes[vertex] = Nodes[vertex][1:]
                    
                #Lista degli Amino acidi
                    #Amino = list(proteina.iloc[0][1:])
                Amino = amino.transform(list(proteina.iloc[0][1:]))

                
                proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                j = proteina.iloc[0]
                proteina.drop([0], axis = 0, inplace = True)
                
                
                    #proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                    #proteina.drop([0], axis = 0, inplace = True)
                #Classe della proteina
                classe = proteine[proteine['Id'] == id_p]["Classe"].iloc[0][0]
        
                
                #Creazione Grafo
                G = nx.Graph()
                G.add_nodes_from(Nodes)
                
                #Aggiunta archi
                rows, cols = proteina.shape
                Vprimo = "Vx"
                Vsecondo ="Vx"
                for i in range(rows):
                    row = proteina.iloc[i]
                    j = 0
                    for edge in row:
                        if edge == '1':
                            vPrimo = Vprimo[:1] + str(i+1)
                            vSecondo = Vsecondo[:1] + str(j+1)
                            #print("Aggiunta arco tra il nodo ", vPrimo[1:],"e il nodo ", vSecondo[1:])
                            G.add_edge(vPrimo[1:], vSecondo[1:])
                        j += 1
                
                #Conbersione Grafo in formato DGL
                new_g = dgl.from_networkx(G)
                
                
                
                #Creazione file CSV Grafi
                id_graph = ID_Graphs_read.transform([id_p])[0]

                a = new_g.edges()[0]
                b = new_g.edges()[1]
                src_id = a.tolist()
                dst_id = b.tolist()
                    
                
                graph_id = []
                for i in range(0,len(src_id)):
                    graph_id.append(id_graph)
                
                """# Create dict edge
                data = {'graph_id': graph_id,
                        'src_id': src_id,
                        'dst_id': dst_id
                       }

                # Create DataFrame
                edge = pd.DataFrame(data)
                
                edges = edges.merge(edge, how='outer')
                """
                
                #Inserimento informazioni archi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], src_id[i], dst_id[i]]
                    writer_e.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_e
                # f_e.close()
                
                
                node = new_g.nodes()
                node_id = node.tolist()
                graph_id = graph_id[:len(Amino)]

                
                Amino = Amino.astype("float")
                Amino = Amino.astype("str")
                
                """# Create dict node
                data = {'graph_id': graph_id,
                        'node_id': node_id,
                        'label': Amino
                       }
                # Create DataFrame
                node = pd.DataFrame(data)
                nodes = nodes.merge(node, how = 'outer')"""
                
                #Questa porzione di codice serve per introdurre la virgola nei file
                end = ",0.0" #si può levare lo 0 solo che potrebbero esserci conflitti
                for i in range(0,len(Amino)):
                    Amino[i] =Amino[i]+end
                #######
                
                
                #Inserimento informazioni nodi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], node_id[i], Amino[i]]
                    writer_n.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_n
                # f_n.close()
                
        
                classe = classi.transform([classe])
                graph_id = graph_id[:1]

                
                classe = classe.astype("float")
                print(classe)
                """ # Create dict graph
                data = {'graph_id': graph_id,
                        'label': classe
                       }

                # Create DataFrame
                graph = pd.DataFrame(data)
                
                graphs = graphs.merge(graph, how = 'outer')"""
            
                #Inserimento informazioni nodi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], classe[i]]
                    writer_g.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_g 
                # f_g.close()
                
#edges.to_csv('../Dati/Dataset/edges.csv', index=False)
#nodes.to_csv('../Dati/Dataset/nodes.csv', index=False)
#graphs.to_csv('../Dati/Dataset/graphs.csv', index=False)


f_e.close()
f_n.close()
f_g.close()

3fb5
[1.]
5ec2
[1.]
4vhb
[0.]
1ri8
[1.]
1cdu
[1.]
1jl6
[0.]
1or4
[0.]
1cdb
[1.]
2qfn
[0.]
1dzb
[1.]
4uuj
[1.]
1jtp
[1.]
1cid
[1.]
1dly
[0.]
1ash
[0.]
1s61
[0.]
1jl7
[0.]
1cdc
[1.]
1rjc
[1.]
2g75
[1.]
3ord
[0.]
1ch4
[0.]
1h8n
[1.]
1or6
[0.]
3ok5
[0.]
3oj1
[0.]
1rsf
[1.]
3or7
[1.]
1vrf
[0.]
1ci5
[1.]
1qok
[1.]
2atp
[1.]
1s6a
[0.]
1ncn
[1.]
1uvx
[0.]
1ieh
[1.]
3cd4
[1.]
2gl3
[0.]
2qfk
[0.]
1jma
[1.]
1op9
[1.]
1mel
[1.]
1vre
[0.]
1jtt
[1.]
1f5w
[1.]
1ut0
[0.]
1uvy
[0.]
1urv
[0.]
1g9e
[1.]
3dr9
[0.]
5yxk
[1.]
4hsx
[0.]
1sjv
[1.]
5vke
[1.]
2hbg
[0.]
4ilz
[0.]
1ol0
[1.]
3gb7
[1.]
4max
[0.]
2giy
[1.]
1eaj
[1.]
2ih3
[1.]
2ih1
[1.]
1it2
[0.]
3tm3
[0.]
1jf4
[0.]
2vhb
[0.]
1zmy
[1.]
1eca
[0.]
1it3
[0.]
1rte
[0.]
1i8l
[1.]
1il1
[1.]
1ew6
[0.]
1wio
[1.]
3tld
[0.]
2hz1
[0.]
1x9f
[0.]
1ecd
[0.]
1ewa
[0.]
1jf3
[0.]
5vk6
[1.]
1v5h
[0.]
1zv5
[1.]
1dr9
[1.]
2hz2
[0.]
2hz3
[0.]
2gkn
[0.]
3tm9
[0.]
2hvk
[1.]
1zvy
[1.]
4l2m
[0.]
1ccz
[1.]
5v3v
[0.]
4gzg
[0.]
1gvh
[0.]
1rtx
[0.]
4fh6
[0.]
2gkm
[0.]
4kmv
[0.]
